In [1]:
# importing necessary Python libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
#import plotly.offline as pyoff
import plotly.graph_objs as go 
#import plotly.figure_factory as ff

# avoid displaying warnings
import warnings
warnings.filterwarnings("ignore")

#import machine learning related libraries
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, cross_validate

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.cluster import KMeans
import xgboost as xgb
import time 

# Loading the data
df = pd.read_csv('online_retail_II.csv')

DATA WRANGLING


In [2]:
# Rename the following columns: 
#    Invoice to InvoiceNo
#    Customer ID to CustomerID
#    Price to UnitPrice

df.rename(columns={'Invoice':'InvoiceNo', 'Customer ID':'CustomerID', 
                   'Price':'UnitPrice'}, 
          inplace=True)

Dropping empty CustomerID

In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824364 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    824364 non-null  object 
 1   StockCode    824364 non-null  object 
 2   Description  824364 non-null  object 
 3   Quantity     824364 non-null  int64  
 4   InvoiceDate  824364 non-null  object 
 5   UnitPrice    824364 non-null  float64
 6   CustomerID   824364 non-null  float64
 7   Country      824364 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 56.6+ MB


In [5]:
df.InvoiceDate=pd.to_datetime(df.InvoiceDate)

In [7]:
ctm_bhvr_dt = df[(df.InvoiceDate < pd.Timestamp(2011,9,1)) & (df.InvoiceDate >= pd.Timestamp(2009,12,1))].reset_index(drop=True)
ctm_bhvr_dt.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [8]:
ctm_next_quarter = df[(df.InvoiceDate < pd.Timestamp(2011,12,1)) & (df.InvoiceDate >= pd.Timestamp(2011,9,1))].reset_index(drop=True)
ctm_next_quarter

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,565080,20677,PINK POLKADOT BOWL,8,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
1,565080,22128,PARTY CONES CANDY ASSORTED,24,2011-09-01 08:25:00,1.25,13509.0,United Kingdom
2,565082,22423,REGENCY CAKESTAND 3 TIER,2,2011-09-01 09:15:00,12.75,13305.0,United Kingdom
3,565082,15060B,FAIRY CAKE DESIGN UMBRELLA,8,2011-09-01 09:15:00,3.75,13305.0,United Kingdom
4,565082,23245,SET OF 3 REGENCY CAKE TINS,4,2011-09-01 09:15:00,4.95,13305.0,United Kingdom
...,...,...,...,...,...,...,...,...
157110,C579886,22197,POPCORN HOLDER,-1,2011-11-30 17:39:00,0.85,15676.0,United Kingdom
157111,C579886,23146,TRIPLE HOOK ANTIQUE IVORY ROSE,-1,2011-11-30 17:39:00,3.29,15676.0,United Kingdom
157112,C579887,84946,ANTIQUE SILVER T-LIGHT GLASS,-1,2011-11-30 17:42:00,1.25,16717.0,United Kingdom
157113,C579887,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,-1,2011-11-30 17:42:00,7.95,16717.0,United Kingdom


In [13]:
# Get the distinct customers in the dataframe ctm_bhvr_dt
cust_dt = pd.DataFrame(ctm_bhvr_dt['CustomerID'].unique())

# Rename the column to CustomerID.
cust_dt.columns = ['CustomerID']

In [14]:
cust_dt

,CustomerID
0,13085.0
1,13078.0
2,15362.0
3,18102.0
4,12682.0
...,...
5309,15866.0
5310,14660.0
5311,13726.0
5312,15690.0


In [15]:
# Create a dataframe with CustomerID and customers first purchase 
# date in the dataset ctm_next_quarter
ctm_1st_purchase_in_next_quarter = pd.DataFrame(ctm_next_quarter.groupby('CustomerID').InvoiceDate.min()).reset_index()
ctm_1st_purchase_in_next_quarter.columns = ['CustomerID', 'MinPurchaseDate']


In [16]:
ctm_1st_purchase_in_next_quarter

,CustomerID,MinPurchaseDate
0,12347.0,2011-10-31 12:25:00
1,12348.0,2011-09-25 13:13:00
2,12349.0,2011-11-21 09:51:00
3,12352.0,2011-09-20 14:34:00
4,12356.0,2011-11-17 08:40:00
...,...,...
2867,18276.0,2011-10-27 10:54:00
2868,18277.0,2011-10-12 15:22:00
2869,18278.0,2011-09-27 11:58:00
2870,18283.0,2011-09-05 12:35:00


In [17]:
# Create a dataframe with CustomerID and customers last purchase 
# date in the dataset ctm_bhvr_dt
ctm_last_purchase_bhvr_dt = ctm_bhvr_dt.groupby('CustomerID').InvoiceDate.max().reset_index()
ctm_last_purchase_bhvr_dt.columns = ['CustomerID', 'MaxPurchaseDate']


In [18]:
ctm_last_purchase_bhvr_dt

,CustomerID,MaxPurchaseDate
0,12346.0,2011-01-18 10:17:00
1,12347.0,2011-08-02 08:48:00
2,12348.0,2011-04-05 10:47:00
3,12349.0,2010-10-28 08:23:00
4,12350.0,2011-02-02 16:01:00
...,...,...
5309,18283.0,2011-07-14 13:20:00
5310,18284.0,2010-10-06 12:31:00
5311,18285.0,2010-02-17 10:24:00
5312,18286.0,2010-08-20 11:57:00


In [19]:
# Merge two dataframes ctm_last_purchase_bhvr_dt and ctm_1st_purchase_in_next_quarter
ctm_purchase_dates = pd.merge(ctm_last_purchase_bhvr_dt, ctm_1st_purchase_in_next_quarter, on='CustomerID', how='left')

In [20]:
# Get the difference in days from MinPurchaseDate and MaxPurchaseDate for each customer
ctm_purchase_dates['NextPurchaseDay'] = (ctm_purchase_dates['MinPurchaseDate'] - ctm_purchase_dates['MaxPurchaseDate']).dt.days

In [21]:

cust_dt = pd.merge(cust_dt, ctm_purchase_dates[['CustomerID', 'NextPurchaseDay']], on='CustomerID', how='left')
# Fill all missing values in the dataset cust_dt with the number 9999
cust_dt = cust_dt.fillna(9999)
cust_dt.head()

,CustomerID,NextPurchaseDay
0,13085.0,9999.0
1,13078.0,13.0
2,15362.0,9999.0
3,18102.0,27.0
4,12682.0,15.0


In [22]:
ctm_max_purchase = ctm_bhvr_dt.groupby('CustomerID').InvoiceDate.max().reset_index()
ctm_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
ctm_max_purchase.max()


CustomerID                     18287.0
MaxPurchaseDate    2011-08-31 17:45:00
dtype: object

In [23]:
# Find the recency in days 
ctm_max_purchase['Recency'] = (ctm_max_purchase['MaxPurchaseDate'].max() - ctm_max_purchase['MaxPurchaseDate']).dt.days

# Merge the dataframes cust_dt and ctm_max_purchase[['CustomerID', 'Recency']] on the CustomerID column.
cust_dt = pd.merge(cust_dt, ctm_max_purchase[['CustomerID', 'Recency']], on='CustomerID')
cust_dt.head()



,CustomerID,NextPurchaseDay,Recency
0,13085.0,9999.0,57
1,13078.0,13.0,0
2,15362.0,9999.0,348
3,18102.0,27.0,26
4,12682.0,15.0,0


In [24]:
pd.DataFrame(cust_dt.Recency.describe())

,Recency
count,5314.000000
mean,204.675574
std,173.569622
min,0.000000
25%,48.000000
50%,161.000000
75%,320.000000
max,638.000000


Chotu Visualistion

In [26]:
hist_fig = px.histogram(cust_dt,x="Recency",title='Customer Recency in Days',template= "plotly_dark")

hist_fig.update_layout(title_x=0.5,xaxis_title="Recency in groups of 20 days",yaxis_title="No of Custs")

hist_fig.show(config={'displaylogo': False})

In [76]:
my_dict={}
ctm_recency = cust_dt[['Recency']]
for idx in range(1, 10):
    kmeans = KMeans(n_clusters=idx, max_iter=1000).fit(ctm_recency)
    ctm_recency["clusters"] = kmeans.labels_
    my_dict[idx] = kmeans.inertia_ 

line_fig = px.line(x=list(my_dict.keys()), 
                   y=list(my_dict.values()), 
                   template="plotly_dark"
                  )

line_fig.update_layout(title_x=0, 
                       xaxis_title="Number of cluster", 
                       yaxis_title=""
                      )

line_fig.show(config={'displaylogo': False})

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fef76d120e0>
Traceback (most recent call last):
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<loc

In [28]:
number_of_clusters = 4

In [29]:
cust_dt['Recency']

0        57
1         0
2       348
3        26
4         0
       ... 
5309      1
5310      0
5311      0
5312      0
5313      0
Name: Recency, Length: 5314, dtype: int64

In [30]:
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(cust_dt[['Recency']])
cust_dt['RecencyCluster'] = kmeans.predict(cust_dt[['Recency']])

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f46beea7c70>
Traceback (most recent call last):
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f46beea7c70>
Traceback (most recent call last):
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<loc

In [31]:
cust_dt

,CustomerID,NextPurchaseDay,Recency,RecencyCluster
0,13085.0,9999.0,57,2
1,13078.0,13.0,0,2
2,15362.0,9999.0,348,3
3,18102.0,27.0,26,2
4,12682.0,15.0,0,2
...,...,...,...,...
5309,15866.0,9999.0,1,2
5310,14660.0,14.0,0,2
5311,13726.0,61.0,0,2
5312,15690.0,9999.0,0,2


In [37]:
cust_dt.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,1067.0,158.486410,38.720974,100.0,125.0,156.0,189.0,236.0
1,750.0,520.509333,65.218473,418.0,465.0,517.0,571.5,638.0
2,2078.0,39.198749,28.732667,0.0,13.0,34.0,63.0,99.0
3,1419.0,314.801973,40.820649,237.0,282.0,308.0,338.0,416.0


In [39]:
cust_freq=df.groupby('CustomerID').InvoiceDate.count().reset_index()
cust_freq.columns=['CustomerID','Frequency']
cust_freq = pd.merge(cust_dt,cust_freq,on='CustomerID')
cust_freq.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency
0,13085.0,9999.0,57,2,92
1,13078.0,13.0,0,2,855
2,15362.0,9999.0,348,3,40
3,18102.0,27.0,26,2,1068
4,12682.0,15.0,0,2,1042


In [42]:
hist_fig = px.histogram(x=cust_freq.query("Frequency<1200")['Frequency'],title='Customers with Purchase Frequency',template= "plotly_dark")

hist_fig.update_layout(title_x=0.5,xaxis_title="Customer Frequency Purchase in groups of 20",yaxis_title="No of Custs")

hist_fig.show(config={'displaylogo': False})

In [47]:
my_dict={}
ctm_frequency = cust_freq[['Frequency']]
for idx in range(1, 10):
    kmeans = KMeans(n_clusters=idx, max_iter=1000).fit(ctm_frequency)
    ctm_frequency["clusters"] = kmeans.labels_
    my_dict[idx] = kmeans.inertia_ 

line_fig = px.line(x=list(my_dict.keys()), 
                   y=list(my_dict.values()), 
                   template="plotly_dark"
                  )

line_fig.update_layout(title_x=0, 
                       xaxis_title="Number of cluster", 
                       yaxis_title=""
                      )

line_fig.show(config={'displaylogo': False})

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f46a449e050>
Traceback (most recent call last):
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<loc

In [48]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(cust_freq[['Frequency']])
cust_freq['FrequencyCluster'] = kmeans.predict(cust_freq[['Frequency']])

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f46a449feb0>
Traceback (most recent call last):
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/gesskay/anaconda3/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<loc

In [49]:
cust_freq.head()

,CustomerID,NextPurchaseDay,Recency,RecencyCluster,Frequency,FrequencyCluster
0,13085.0,9999.0,57,2,92,3
1,13078.0,13.0,0,2,855,4
2,15362.0,9999.0,348,3,40,3
3,18102.0,27.0,26,2,1068,4
4,12682.0,15.0,0,2,1042,4


In [50]:
cust_freq.groupby('FrequencyCluster')['Frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,865.0,350.165318,123.003356,204.0,251.0,313.0,419.0,691.0
1,2.0,12355.000000,1049.346463,11613.0,11984.0,12355.0,12726.0,13097.0
2,13.0,4035.692308,1540.536616,2548.0,2837.0,3438.0,4717.0,7307.0
3,4275.0,58.348772,51.163292,1.0,18.0,42.0,87.0,203.0
4,159.0,1037.276730,318.910346,694.0,804.5,963.0,1156.5,2430.0
